# Desafío - Transformación y manipulación de datos (Parte II)

1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un DataFrame independiente.

In [ ]:
#comenzamos la carga de las hojas de calculo de excel desde el archivo

import pandas as pd

#cargamos la hoja
file_path = '/content/US_Regional_Sales_Data.xlsx'
xls = pd.ExcelFile(file_path)

#obtenemos los nombres de las hojas
sheet_names = xls.sheet_names

# Create a dictionary to store DataFrames for each sheet
df_sheets = {}
for sheet_name in sheet_names:
    df_sheets[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name)

#mostramos los nombres de las hojas
print(sheet_names)

['Sales Orders Sheet', 'Customers Sheet', 'Store Locations Sheet', 'Products Sheet', 'Sales Team Sheet']


2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado
en una nueva variable llamada df_base.

In [ ]:
# Cruce entre Sales Orders y Customers
df_base = pd.merge(df_sheets['Sales Orders Sheet'], df_sheets['Customers Sheet'], how='left', left_on='_CustomerID', right_on='_CustomerID', validate='many_to_one')

# Cruce con Store Locations
df_base = pd.merge(df_base, df_sheets['Store Locations Sheet'], how='left', left_on='_StoreID', right_on='_StoreID', validate='many_to_one')

# Cruce con Products
df_base = pd.merge(df_base, df_sheets['Products Sheet'], how='left', left_on='_ProductID', right_on='_ProductID', validate='many_to_one')

# Cruce con Sales Team
df_base = pd.merge(df_base, df_sheets['Sales Team Sheet'], how='left', left_on='_SalesTeamID', right_on='_SalesTeamID', validate='many_to_one')

# Ver una muestra de los datos finales cruzados
df_base.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,AreaCode,Population,Household Income,Median Income,Land Area,Water Area,Time Zone,Product Name,Sales Team,Region
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,631,213776,68789,80327,135481314,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,913,186515,74830,72463,194562941,1309517,America/Chicago,Wreaths,Paul Holmes,Midwest
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,734,117070,47179,55990,72727009,2254528,America/Detroit,Stemware,Samuel Fowler,Midwest
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,203,130322,49771,37192,48407373,3739487,America/New York,Accessories,Carlos Miller,West
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,203,108802,40213,40467,73880017,1086045,America/New York,Candles,Joe Price,Northeast


3. Agrega las columnas calculadas al df

In [ ]:
df_base['ProcuredDate'] = pd.to_datetime(df_base['ProcuredDate'])
df_base['OrderDate'] = pd.to_datetime(df_base['OrderDate'])
df_base['ShipDate'] = pd.to_datetime(df_base['ShipDate'])
df_base['DeliveryDate'] = pd.to_datetime(df_base['DeliveryDate'])

# Calcular ProcurementDays (OrderDate - ProcuredDate)
df_base['ProcurementDays'] = (df_base['OrderDate'] - df_base['ProcuredDate']).dt.days

# Calcular ShippingDays (ShipDate - OrderDate)
df_base['ShippingDays'] = (df_base['ShipDate'] - df_base['OrderDate']).dt.days

# Calcular DeliveryDays (DeliveryDate - ShipDate)
df_base['DeliveryDays'] = (df_base['DeliveryDate'] - df_base['ShipDate']).dt.days

# Calcular CustomerDays (ShippingDays + DeliveryDays)
df_base['CustomerDays'] = df_base['ShippingDays'] + df_base['DeliveryDays']

# Ver los resultados
df_base[['ProcurementDays', 'ShippingDays', 'DeliveryDays', 'CustomerDays']].head()


,ProcurementDays,ShippingDays,DeliveryDays,CustomerDays
0,151,14,5,19
1,151,22,10,32
2,151,21,10,31
3,151,2,5,7
4,51,16,10,26


In [ ]:
# verifico que sean de tipo datetime
print(df_base.dtypes[['ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate']])

ProcuredDate    datetime64[ns]
OrderDate       datetime64[ns]
ShipDate        datetime64[ns]
DeliveryDate    datetime64[ns]
dtype: object


4. Agrega una nueva columna discreta al df_base, con el nombre CustomerDaysInterval
que clasifique los valores de la columna CustomerDays

In [ ]:
# Definir los intervalos y etiquetas
bins = [0, 15, 30, 45, 60, 75, 90]
labels = ['0 to 15 days', '15 to 30 days', '30 to 45 days', '45 to 60 days', '60 to 75 days', '75 to 90 days']

# Crear la nueva columna CustomerDaysInterval
df_base['CustomerDaysInterval'] = pd.cut(df_base['CustomerDays'], bins=bins, labels=labels)

# Ver una muestra de los resultados
df_base[['CustomerDays', 'CustomerDaysInterval']].head()


,CustomerDays,CustomerDaysInterval
0,19,15 to 30 days
1,32,30 to 45 days
2,31,30 to 45 days
3,7,0 to 15 days
4,26,15 to 30 days


5. Utiliza una tabla pivote para mostrar el conteo de órdenes agrupando por Sales
Team en las filas y CustomerDaysInterval en las columnas

In [ ]:
# Crear la tabla pivote


pivot_table = df_base.pivot_table(
    index='Sales Team',                 # Agrupando por Sales Team
    columns='CustomerDaysInterval',     # Columnas para CustomerDaysInterval
    values='OrderNumber',               # Conteo basado en el número de órdenes
    aggfunc='count',                    # Usar conteo
    fill_value=0,                       # Rellenar con 0 donde no hay datos
    margins=True                        # Mostrar totales
)

# Ver la tabla pivote
pivot_table


<ipython-input-32-278ca3c6cd3e>:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_table = df_base.pivot_table(


CustomerDaysInterval,0 to 15 days,15 to 30 days,30 to 45 days,45 to 60 days,60 to 75 days,75 to 90 days,All
Sales Team,,,,,,,
Adam Hernandez,81,179,42,0,0,0,302
Anthony Berry,76,181,41,0,0,0,298
Anthony Torres,81,162,45,0,0,0,288
Carl Nguyen,96,178,40,0,0,0,314
Carlos Miller,65,148,34,0,0,0,247
Chris Armstrong,92,156,44,0,0,0,292
Donald Reynolds,97,163,36,0,0,0,296
Douglas Tucker,87,145,45,0,0,0,277
Frank Brown,85,150,40,0,0,0,275


6. Agregar columna calculada GrossMargin

In [ ]:
# Calcular la columna GrossMargin
df_base['GrossMargin'] = df_base['Order Quantity'] * (df_base['Unit Price'] * (1 - df_base['Discount Applied']) - df_base['Unit Cost'])

# Ver las primeras filas de la nueva columna GrossMargin
df_base[['OrderNumber', 'GrossMargin']].head()

,OrderNumber,GrossMargin
0,SO - 000101,4073.4325
1,SO - 000102,886.4100
2,SO - 000103,905.5050
3,SO - 000104,5486.7640
4,SO - 000105,1312.1280


7. Aplicar un porcentaje a la columna GrossMargin,

In [ ]:
# Definir los intervalos y los porcentajes de comisión
bins = [0, 100, 1000, 10000, 100000, float('inf')]
labels = [0.05, 0.10, 0.15, 0.20, 0.25]

# Crear la columna CommissionsPercentage usando pd.cut
df_base['CommissionsPercentage'] = pd.cut(
    df_base['GrossMargin'],
    bins=bins,
    labels=labels,
    right=False
)

# Convertir a tipo float, ignorando NaNs si hay algún valor fuera de los intervalos
df_base['CommissionsPercentage'] = df_base['CommissionsPercentage'].astype(float)

# Ver las primeras filas para verificar el resultado
print(df_base[['GrossMargin', 'CommissionsPercentage']].head())

   GrossMargin  CommissionsPercentage
0    4073.4325                   0.15
1     886.4100                   0.10
2     905.5050                   0.10
3    5486.7640                   0.15
4    1312.1280                   0.15


8. Calcula el monto de la comisión

In [ ]:
# Calcular la columna CommissionsAmount
df_base['CommissionsAmount'] = df_base['GrossMargin'] * df_base['CommissionsPercentage']

# Ver las primeras filas para verificar el resultado
print(df_base[['GrossMargin', 'CommissionsPercentage', 'CommissionsAmount']].head())

   GrossMargin  CommissionsPercentage  CommissionsAmount
0    4073.4325                   0.15         611.014875
1     886.4100                   0.10          88.641000
2     905.5050                   0.10          90.550500
3    5486.7640                   0.15         823.014600
4    1312.1280                   0.15         196.819200


9. Calcula la comisión sobre el margen bruto.

In [ ]:
# Calcular la columna NetMargin
df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount']

# Ver las primeras filas para verificar el resultado
print(df_base[['GrossMargin', 'CommissionsAmount', 'NetMargin']].head())

   GrossMargin  CommissionsAmount    NetMargin
0    4073.4325         611.014875  3462.417625
1     886.4100          88.641000   797.769000
2     905.5050          90.550500   814.954500
3    5486.7640         823.014600  4663.749400
4    1312.1280         196.819200  1115.308800


10. Utilizando una tabla pivote, muestra la suma de los valores GrossMargin, NetMargin y
CommissionAmount agrupando por Sales Team en las filas.

In [ ]:
# Crear la tabla pivote
pivot_table = df_base.pivot_table(
    index='Sales Team',                      # Agrupar por Sales Team
    values=['GrossMargin', 'NetMargin', 'CommissionsAmount'],  # Valores a sumar
    aggfunc='sum',                           # Usar suma
    fill_value=0                             # Rellenar con 0 donde no hay datos
)

# Ver la tabla pivote
pivot_table

,CommissionsAmount,GrossMargin,NetMargin
Sales Team,,,
Adam Hernandez,126637.207525,779095.1955,677987.332975
Anthony Berry,124061.181475,767943.9500,654798.341525
Anthony Torres,117962.211675,743901.4355,636758.048825
Carl Nguyen,123843.212050,774781.4675,661874.732450
Carlos Miller,110882.093875,668752.9290,580326.019125
Chris Armstrong,122565.707975,781608.7005,660857.419525
Donald Reynolds,143768.338700,885386.8095,754038.260800
Douglas Tucker,120806.403550,745528.5640,633786.054450
Frank Brown,104839.466050,670881.2510,570076.792950
